In [115]:
import pandas as pd
import glob
import re
import unicodedata
import numpy as np
import ast 

Nesta secção, vamos juntar todos os produtos extraidos  de cada um dos sites seja em black friday ou não, vamos criar um dataset com todos os produtos e respectivas informações. Depois vamos fazer uma limpeza dos dados repetidos e por fim aplicar um modelo de agregação e categorização dos produtos.

In [ ]:
file_paths = glob.glob("data/*products_extracted_prices*.csv") # with glob we can get all the files that match the pattern
dataframes = []

for file_path in file_paths:
    df = pd.read_csv(file_path)
    
    if "Black_Friday" in file_path:
        df['promocao'] = 'Black Friday'
    else:
        df['promocao'] = 'Sem Promocao'
    
    dataframes.append(df)

merged_df = pd.concat(dataframes, ignore_index=True)

merged_df.to_csv("all_prices_extracted.csv", index=False)

Abaixo vou so fazer um teste para ter a certeza que o codigo acima funcionou como pretendido.

In [18]:
file_paths = glob.glob("data/*products_extracted_prices*.csv") 

black_friday_counts = {}
not_black_friday_counts = {}

for file_path in file_paths:
    if "Black_Friday" in file_path:
        df = pd.read_csv(file_path)
        black_friday_counts[file_path] = df.shape[0]
    else:
        df = pd.read_csv(file_path)
        not_black_friday_counts[file_path] = df.shape[0]
        
total_black_friday_rows = sum(black_friday_counts.values())
print(f"Total Black Friday rows across all files: {total_black_friday_rows}")
total_black_friday_rows_on_merged_df = merged_df[merged_df['promocao'] == 'Black Friday'].shape[0]
print(f"Total Black Friday rows on merged DataFrame: {total_black_friday_rows_on_merged_df}")

total_not_black_friday_rows = sum(not_black_friday_counts.values())
print(f"Total Not Black Friday rows across all files: {total_not_black_friday_rows}")
total_not_black_friday_rows_on_merged_df = merged_df[merged_df['promocao'] == 'Sem Promocao'].shape[0]
print(f"Total Not Black Friday rows on merged DataFrame: {total_not_black_friday_rows_on_merged_df}")

Total Black Friday rows across all files: 839
Total Black Friday rows on merged DataFrame: 839
Total Not Black Friday rows across all files: 703
Total Not Black Friday rows on merged DataFrame: 703


In [117]:
merged_df = pd.read_csv("all_prices_extracted.csv")
merged_df = merged_df.drop(columns=['linkToArchive','linkToExtractedText','snippet'])

In [118]:
merged_df.isna().sum()


site              0
date              0
title            16
extractedData    55
promocao          0
dtype: int64

In [119]:
# eliminate rows with missing values
merged_df = merged_df.dropna()
merged_df.isna().sum()

site             0
date             0
title            0
extractedData    0
promocao         0
dtype: int64

In [120]:
print(f"Shape before: {merged_df.shape}")

merged_df = merged_df.drop_duplicates()

print(f"Shape after: {merged_df.shape}")


Shape before: (1476, 5)
Shape after: (967, 5)


In [ ]:
def clean_extracted_data(extracted_str):
    try:
        extracted_list = ast.literal_eval(extracted_str) # é suado para converter as strings em listas
    except (ValueError, SyntaxError):
        return []  

    cleaned_prices = []
    for price in extracted_list:
        normalized_price = unicodedata.normalize("NFKD", price)
        cleaned_price = re.sub(r'[^\d,]', '', normalized_price)
        cleaned_prices.append(cleaned_price)
    
    return cleaned_prices

merged_df['extractedData'] = merged_df['extractedData'].apply(clean_extracted_data)


In [122]:
merged_df.to_csv("all_prices_extracted_cleaned.csv", index=False)